In [1]:
"""Find paths explaining connections between phenotypes and their nearest genes.

METSIM Metabolomics PheWeb
https://pheweb.org/metsim-metab/
"""

import pandas as pd

import gilda
from indra.databases import hgnc_client
import bioregistry

URL = "https://pheweb.org/metsim-metab/download/phenotypes.tsv"

COLUMNS = {
    "chrom": "chromosome",
    "pos": "position",
    "ref": "reference_allele",
    "alt": "alternative_allele",
    "rsids": "dbsnp_ids",  # comma separated
    "nearest_genes": "nearest_gene_symbols",  # comma separated
    "pval": "p_value",
    "num_peaks": "number_peaks",
    "phenocode": "identifier",  # this is the primary identifier, e.g., C100001054
    "category": "category",
    "phenostring": "label",  # this is the label for the entry, e.g., butyrylcarnitine (C4)
}

NOTED_MISSING = set()

def _norm_genes(symbols):
    rv = []
    for symbol in symbols:
        hgnc_id = hgnc_client.get_current_hgnc_id(symbol)
        if hgnc_id is None:
            if symbol not in NOTED_MISSING:
                print(f"could not look up HGNC id for {symbol}")
                NOTED_MISSING.add(symbol)
            continue
        rv.append(hgnc_id)
    return rv

df = pd.read_csv(URL, sep="\t").rename(columns=COLUMNS)
df = df[df.category != "Uncharacterized"]
df = df[["identifier", "label", "category", "dbsnp_ids", "nearest_gene_symbols"]]
for column in "dbsnp_ids", "nearest_gene_symbols":
    df[column] = df[column].map(lambda s: s.split(",") if pd.notna(s) else [])
df["nearest_gene_ids"] = df["nearest_gene_symbols"].map(_norm_genes)
del df["nearest_gene_symbols"]
df.head()

could not look up HGNC id for AC068946.1
could not look up HGNC id for AC007326.4
could not look up HGNC id for AC008012.1
could not look up HGNC id for AP000356.5
could not look up HGNC id for AL360181.3
could not look up HGNC id for AL022238.3
could not look up HGNC id for AC020765.6
could not look up HGNC id for AC097104.1
could not look up HGNC id for AC004832.3
could not look up HGNC id for AL583836.1
could not look up HGNC id for AC008537.1
could not look up HGNC id for AL355312.5
could not look up HGNC id for AL355315.1
could not look up HGNC id for AC027644.4
could not look up HGNC id for FO393400.1
could not look up HGNC id for AC069288.1
could not look up HGNC id for AC008695.1
could not look up HGNC id for AC007375.2
could not look up HGNC id for AL162596.1
could not look up HGNC id for AC010197.2


,identifier,label,category,dbsnp_ids,nearest_gene_ids
0,C100001054,butyrylcarnitine (C4),Lipid,[rs2066938],[16488]
1,C100001104,N-acetyltyrosine,Amino_Acid,[rs6546861],[428]
2,C100001253,N-acetylglutamine,Amino_Acid,[rs10207220],[428]
3,C100001266,N-acetylarginine,Amino_Acid,"[rs142350660, rs376360741]",[428]
4,C100001402,5-acetylamino-6-formylamino-3-methyluracil,Xenobiotics,[rs34370554],[7646]


In [2]:
def ground(t):
    t = t.rstrip("*")
    for i in range(5):
        t = t.removesuffix(f" ({i})")

    # could do additional string processing for examples like:
    #  - (14 or 15)-methylpalmitate (a17:0 or i17:0)
    
    x = gilda.ground(t)
    if x:
        term = x[0].term
        return term.db, term.id, term.entry_name
    
    if t.endswith("ate"):
        new_t = t[:-len("ate")] + "ic acid"
        x = gilda.ground(new_t)
        if x:
            term = x[0].term
            return term.db, term.id, term.entry_name
        
    if t.endswith(" sulfate"):
        new_t = t.removesuffix(" sulfate")
        x = gilda.ground(new_t)
        if x:
            term = x[0].term
            return term.db, term.id, term.entry_name

    return None

df["grounding"] = df.label.map(ground)
grounding_idx = df["grounding"].notna()
print(f"successfully grounded {grounding_idx.sum()} rows")
grounded_df = df[grounding_idx]
grounded_df.head()

successfully grounded 597 rows


,identifier,label,category,dbsnp_ids,nearest_gene_ids,grounding
1,C100001104,N-acetyltyrosine,Amino_Acid,[rs6546861],[428],"(CHEBI, CHEBI:68561, N-acetyltyrosine)"
2,C100001253,N-acetylglutamine,Amino_Acid,[rs10207220],[428],"(MESH, C032007, aceglutamide)"
4,C100001402,5-acetylamino-6-formylamino-3-methyluracil,Xenobiotics,[rs34370554],[7646],"(MESH, C036692, 5-acetylamino-6-formylamino-3-..."
5,C100001466,3-methylcytidine,Nucleotide,[rs17385861],[29253],"(CHEBI, CHEBI:20129, 3-methylcytidine)"
6,C100001577,N-acetylcitrulline,Amino_Acid,[rs10168931],[18069],"(CHEBI, CHEBI:49006, N-acetylcitrulline)"


In [3]:
for label in sorted(df[~grounding_idx].label):
    print(label)

(14 or 15)-methylpalmitate (a17:0 or i17:0)
(16 or 17)-methylstearate (a19:0 or i19:0)
(2 or 3)-decenoate (10:1n7 or n8)
(2,4 or 2,5)-dimethylphenol sulfate
(N(1) + N(8))-acetylspermidine
(S)-3-hydroxybutyrylcarnitine
(S)-a-amino-omega-caprolactam
1,2-dilinoleoyl-GPC (18:2/18:2)
1,2-dipalmitoyl-GPC (16:0/16:0)
1,2-distearoyl-GPC (18:0/18:0)
1,5-anhydroglucitol (1,5-AG)
1-(1-enyl-oleoyl)-2-docosahexaenoyl-GPE (P-18:1/22:6)*
1-(1-enyl-oleoyl)-GPE (P-18:1)*
1-(1-enyl-palmitoyl)-2-oleoyl-GPC (P-16:0/18:1)*
1-(1-enyl-palmitoyl)-2-oleoyl-GPE (P-16:0/18:1)*
1-(1-enyl-palmitoyl)-GPC (P-16:0)*
1-(1-enyl-stearoyl)-2-arachidonoyl-GPC (P-18:0/20:4)
1-(1-enyl-stearoyl)-2-arachidonoyl-GPE (P-18:0/20:4)*
1-(1-enyl-stearoyl)-2-docosahexaenoyl-GPE (P-18:0/22:6)*
1-(1-enyl-stearoyl)-2-oleoyl-GPE (P-18:0/18:1)
1-arachidonoyl-GPC (20:4n6)*
1-arachidonoyl-GPE (20:4n6)*
1-arachidonylglycerol (20:4)
1-carboxyethylisoleucine
1-carboxyethylleucine
1-carboxyethylphenylalanine
1-carboxyethyltyrosine
1-carboxyeth

In [6]:
queries = []
for (prefix, identifier, name), genes in grounded_df[["grounding", "nearest_gene_ids"]].values:
    norm_prefix, norm_id = bioregistry.normalize_parsed_curie(prefix, identifier)
    for gene in genes:
        queries.append((
            "hgnc", gene, hgnc_client.get_hgnc_name(gene),
            norm_prefix, norm_id, name,
        ))
columns = [
    "gene_prefix",
    "gene_identifier",
    "gene_name",
    "phenotype_prefix",
    "phenotype_identifier",
    "phenotype_name",
]
queries_df = pd.DataFrame(queries, columns=columns).sort_values(columns)
queries_df.to_csv("pheweb_metsim_tests.tsv", sep="\t", index=False)
queries_df.head()

,gene_prefix,gene_identifier,gene_name,phenotype_prefix,phenotype_identifier,phenotype_name
633,hgnc,10348,RPL37A,chebi,27592,ectoine
350,hgnc,1048,BHMT2,chebi,90344,N-methylproline
39,hgnc,1048,BHMT2,mesh,C025138,dimethylglycine
100,hgnc,10487,S100A10,chebi,132979,3-hydroxydecanoate
141,hgnc,10487,S100A10,chebi,133511,3-hydroxyoctanoate
